# Notebook for Reorganizing Data in Coding DH

Realized we needed to improve how we archived our data so that we could get historic snapshots. This notebook is for reorganizing the data.

In [ ]:
import os
import pandas as pd

In [ ]:


current_path = "../../datasets/repo_data"
older_path = "../../datasets/older_files/repo_data"
file_paths_dfs = []

for dir, _, files in os.walk(current_path):
    older_dir = dir.replace(current_path, older_path)
    if os.path.exists(older_dir):
        older_files = os.listdir(older_dir)
        for file in files:
            subset_file = os.path.splitext(file)[0]
            for older_file in older_files:
                if subset_file in older_file:
                    file_dict = {
                        'file_path': os.path.join(dir, file),
                        'subset_file': subset_file,
                        'older_file_path': os.path.join(older_dir, older_file)
                    }
                    file_paths_dfs.append(file_dict)
files_df = pd.DataFrame(file_paths_dfs)
files_df[0:1].to_dict()